# Yapay Sinir Ağları (YSA)

İnsan beyninin bilgi işleme şeklini referans alan sınıflandırma ve regresyon problemleri için kullanılabilen kuvvetli makine öğrenmesi algoritmalarından birisidir.

Sinir Ağı(Neural Network): Beyinden ilham alan bir model, basit bağlı birimlerden veya nöronlardan ve ardından doğrusal olmayan(en az biri gizli) katmanlardan oluşur. Amaç en küçük hata ile tahmin yapabilecek katsayılara erişmektir.

Yapay Sinir Hücresi(Neuron): Bir sinir ağında, tipik olarak çoklu giriş değerleri alan ve bir çıkış değeri üreten bir düğüm. Nöron, ağırlıklı bir girdi değerleri toplamına bir aktivasyon fonksiyonu (doğrusal olmayan dönüşüm) uygulayarak çıktı değerini hesaplar.

<center><img src="nn.jpg"/><\center>

Aktivasyon Fonksiyonu: Bir önceki katmandaki tüm girdilerin ağırlıklı toplamını alan ve daha sonra bir çıkış değeri (tipik olarak doğrusal olmayan) üreten ve bir sonraki katmana geçiren bir işlev (örneğin, ReLU veya sigmoid).

Yapay sinir ağları birçok yapay sinir hücresinden oluşur.

<center><img src="hidden.png"/><\center>

Gizli Katman: Giriş katmanı (yani özellikler) ve çıkış katmanı (tahmin) arasındaki sinir ağı içindeki sentetik katman. Gizli katmanlar genellikle eğitim için bir aktivasyon fonksiyonu (ReLU gibi) içerir. Derin bir sinir ağı birden fazla gizli katman içerir.

\begin{gather*}
\large h_k(x) = g \bigg( \beta_{0k} + \sum_{j=1}^P x_j \beta_{jk} \bigg)
\end{gather*}

\begin{gather*}
\large g(u) = \frac{1}{1+e^{-u}}
\end{gather*}

\begin{gather*}
\large f(x) = \gamma + \sum_{k=1}^H \gamma_k h_k
\end{gather*}

\begin{gather*}
\large \sum_{i=1}^n (y_i - f_i(x))^2 + \lambda \sum_{k=i}^H \sum_{j=0}^P \beta_{jk}^2 + \lambda \sum_{k=0}^H \gamma_k^2
\end{gather*}

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, ShuffleSplit, GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.neural_network import MLPRegressor
import warnings 
warnings.filterwarnings('ignore')

## Veri Ön İşleme

In [2]:
hitters = pd.read_csv('Hitters.csv')
df = hitters.copy()
df = df.dropna()
dummies = pd.get_dummies(df.select_dtypes(["object"])) 
y = df['Salary']
X_ = df.drop(['Salary', 'League', 'NewLeague', 'Division'], axis=1).astype("float64")
X = pd.concat([X_, dummies[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Model

In [3]:
mlp_model = MLPRegressor().fit(X_train_scaled, y_train)

In [4]:
mlp_model.n_layers_

3

In [5]:
mlp_model.hidden_layer_sizes

(100,)

## Tahmin

In [6]:
mlp_model.predict(X_train_scaled)[0:5]

array([ 4.69216768, 48.49950999, 39.70944554,  3.62451718,  3.12093612])

In [7]:
y_pred = mlp_model.predict(X_test_scaled)
np.sqrt(mean_squared_error(y_test, y_pred))

654.4254045257578

## Tuning

In [8]:
mlpc_params = {'alpha':[1, 5, 0.1, 0.01, 0.03, 0.005, 0.0001 ],
              'hidden_layer_sizes': [(10,10), (100,100,100), (100,100), (3,5)],
              'activation': ['relu', 'logistic']}

In [9]:
mlp_cv = GridSearchCV(mlp_model, mlpc_params, cv=10).fit(X_train_scaled, y_train)

In [10]:
mlp_cv.best_params_

{'activation': 'relu', 'alpha': 1, 'hidden_layer_sizes': (100, 100, 100)}

In [11]:
mlp_tuned =  MLPRegressor(
    alpha= mlp_cv.best_params_['alpha'] , 
    hidden_layer_sizes= mlp_cv.best_params_['hidden_layer_sizes'] , 
    activation= mlp_cv.best_params_['activation']).fit(X_train_scaled, y_train)

In [12]:
y_pred = mlp_tuned.predict(X_test_scaled)
np.sqrt(mean_squared_error(y_test, y_pred))

358.35665624516463

## Kaynaklar

### Websiteleri
* https://developers.google.com/machine-learning/crash-course

### Kurslar
* https://www.udemy.com/course/python-egitimi/